In [2]:
# CELDA 1: FUNCIÓN DE CARGA Y PREPROCESAMIENTO DE DATOS
def load_data():
    """
    Esta función se encarga de cargar y preparar el dataset de enfermedades cardíacas
    para el análisis de machine learning.
    
    Returns:
        tuple: (x, y) donde x son las características y y es la variable objetivo
    """
    
    # Importamos pandas para manipulación de datos
    import pandas as pd

    # Cargamos el dataset desde el archivo CSV ubicado en la carpeta files/input
    # Este dataset contiene información médica de pacientes para predecir enfermedades cardíacas
    dataset = pd.read_csv("../files/input/heart_disease.csv")
    
    # Extraemos la variable objetivo (target) del dataset
    # La variable 'target' indica si el paciente tiene enfermedad cardíaca (1) o no (0)
    # El método pop() remueve la columna del dataframe y la retorna
    y = dataset.pop("target")
    
    # Creamos una copia del dataset sin la variable objetivo
    # Esto contendrá todas las características (features) que usaremos para predecir
    x = dataset.copy()
    
    # Preprocesamos la variable categórica 'thal' (tipo de defecto de talio)
    # Convertimos valores que no sean 'fixed' o 'reversible' a 'normal'
    # Esto normaliza los datos categóricos para mejorar el rendimiento del modelo
    x["thal"] = x["thal"].map(
        lambda x: "normal" if x not in ["fixed", "fixed", "reversible"] else x
    )

    # Retornamos las características (x) y la variable objetivo (y)
    return x, y


# Ejecutamos la función para cargar los datos y los asignamos a variables globales
# x: DataFrame con las características de los pacientes
# y: Serie con las etiquetas de enfermedad cardíaca (0 o 1)
x, y = load_data()

# Mostramos el DataFrame de características para inspeccionar los datos cargados
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,52,1,1,118,186,0,2,190,0,0.0,2,0,fixed
299,43,0,4,132,341,1,2,136,1,3.0,2,0,reversible
300,65,1,4,135,254,0,2,127,0,2.8,2,1,reversible
301,48,1,4,130,256,1,2,150,1,0.0,1,2,reversible


In [3]:
# CELDA 2: FUNCIÓN PARA DIVIDIR LOS DATOS EN ENTRENAMIENTO Y PRUEBA
def make_train_test_split(x, y):
    """
    Esta función divide el dataset en conjuntos de entrenamiento y prueba
    para evaluar el rendimiento del modelo de manera objetiva.
    
    Args:
        x: DataFrame con las características
        y: Serie con las etiquetas objetivo
    
    Returns:
        tuple: (x_train, x_test, y_train, y_test) - conjuntos de entrenamiento y prueba
    """
    
    # Importamos la función para dividir los datos
    from sklearn.model_selection import train_test_split

    # Dividimos los datos en entrenamiento (90%) y prueba (10%)
    # test_size=0.10: destinamos el 10% de los datos para prueba
    # random_state=0: fijamos la semilla aleatoria para reproducibilidad
    # Esto asegura que siempre obtengamos la misma división de datos
    (x_train, x_test, y_train, y_test) = train_test_split(
        x,                    # Características de entrada
        y,                    # Variable objetivo
        test_size=0.10,       # 10% para prueba, 90% para entrenamiento
        random_state=0,       # Semilla para reproducibilidad
    )
    
    # Retornamos los cuatro conjuntos:
    # x_train: características para entrenar el modelo
    # x_test: características para evaluar el modelo
    # y_train: etiquetas para entrenar el modelo  
    # y_test: etiquetas para evaluar el modelo
    return x_train, x_test, y_train, y_test

In [4]:
# CELDA 3: FUNCIÓN PARA CREAR EL PIPELINE DE MACHINE LEARNING
def make_pipeline(estimator):
    """
    Esta función crea un pipeline completo de machine learning que incluye:
    1. Transformación de variables categóricas
    2. Selección de las mejores características
    3. Entrenamiento del modelo
    
    Args:
        estimator: El algoritmo de machine learning a usar (ej: LogisticRegression)
    
    Returns:
        Pipeline: Pipeline completo listo para entrenar
    """
    
    # Importamos las herramientas necesarias de scikit-learn
    from sklearn.compose import ColumnTransformer      # Para transformar columnas específicas
    from sklearn.feature_selection import SelectKBest, f_classif  # Para selección de características
    from sklearn.pipeline import Pipeline             # Para crear el pipeline
    from sklearn.preprocessing import OneHotEncoder   # Para codificar variables categóricas

    # PASO 1: TRANSFORMACIÓN DE DATOS
    # Creamos un transformador que convierte variables categóricas a numéricas
    transformer = ColumnTransformer(
        transformers=[
            # Aplicamos One-Hot Encoding a la columna 'thal' (variable categórica)
            # One-Hot Encoding convierte categorías en columnas binarias (0 o 1)
            # dtype="int": especifica que los valores sean enteros
            ("ohe", OneHotEncoder(dtype="int"), ["thal"]),
        ],
        # remainder="passthrough": mantiene todas las demás columnas sin modificar
        remainder="passthrough",
    )

    # PASO 2: SELECCIÓN DE CARACTERÍSTICAS
    # SelectKBest selecciona las k mejores características basándose en un test estadístico
    # f_classif: usa el test F de ANOVA para problemas de clasificación
    # Esto ayuda a eliminar características irrelevantes y mejorar el rendimiento
    selectkbest = SelectKBest(score_func=f_classif)

    # PASO 3: CREACIÓN DEL PIPELINE
    # Un pipeline ejecuta los pasos secuencialmente:
    # 1. Transforma los datos (One-Hot Encoding)
    # 2. Selecciona las mejores características
    # 3. Entrena el modelo con las características seleccionadas
    pipeline = Pipeline(
        steps=[
            ("tranformer", transformer),    # Transformación de datos categóricos
            ("selectkbest", selectkbest),   # Selección de mejores características
            ("estimator", estimator),       # Algoritmo de machine learning
        ],
        verbose=False,  # No mostrar información detallada durante la ejecución
    )

    # Retornamos el pipeline completo listo para usar
    return pipeline

In [5]:
# CELDA 4: FUNCIÓN PARA BÚSQUEDA DE HIPERPARÁMETROS ÓPTIMOS
def make_grid_search(estimator, param_grid, cv=5):
    """
    Esta función implementa Grid Search para encontrar automáticamente 
    la mejor combinación de hiperparámetros para el modelo.
    
    Args:
        estimator: El modelo/pipeline a optimizar
        param_grid: Diccionario con los parámetros a probar
        cv: Número de folds para validación cruzada (por defecto 5)
    
    Returns:
        GridSearchCV: Objeto configurado para búsqueda de hiperparámetros
    """
    
    # Importamos GridSearchCV para búsqueda exhaustiva de hiperparámetros
    from sklearn.model_selection import GridSearchCV

    # CONFIGURACIÓN DE GRID SEARCH
    # GridSearchCV prueba todas las combinaciones posibles de parámetros
    # y encuentra la que produce el mejor rendimiento usando validación cruzada
    grid_search = GridSearchCV(
        estimator=estimator,              # El pipeline/modelo a optimizar
        param_grid=param_grid,            # Diccionario con parámetros a probar
        cv=cv,                           # Validación cruzada de 5 folds por defecto
        scoring="balanced_accuracy",      # Métrica de evaluación (accuracy balanceada)
    )
    
    # EXPLICACIÓN DE PARÁMETROS:
    # - estimator: El pipeline que queremos optimizar
    # - param_grid: Define qué valores probar para cada parámetro
    # - cv=5: Divide los datos en 5 partes, entrena en 4 y valida en 1, repite 5 veces
    # - scoring="balanced_accuracy": Usa accuracy balanceada (mejor para clases desbalanceadas)

    # Retornamos el objeto GridSearch configurado (aún no ejecutado)
    return grid_search

In [6]:
# CELDA 5: FUNCIÓN PARA GUARDAR EL MODELO ENTRENADO
def save_estimator(estimator):
    """
    Esta función guarda el modelo entrenado en un archivo usando serialización.
    Esto permite reutilizar el modelo sin necesidad de entrenarlo nuevamente.
    
    Args:
        estimator: El modelo entrenado que queremos guardar
    """
    
    # Importamos pickle para serialización de objetos Python
    # Pickle convierte objetos Python en bytes para almacenarlos en archivos
    import pickle

    # PROCESO DE GUARDADO:
    # Abrimos un archivo en modo binario de escritura ("wb")
    # "estimator.pickle" será el nombre del archivo donde se guarda el modelo
    with open("estimator.pickle", "wb") as file:
        # pickle.dump() serializa el objeto estimator y lo guarda en el archivo
        # Esto preserva completamente el estado del modelo entrenado:
        # - Pesos y parámetros aprendidos
        # - Configuración de hiperparámetros
        # - Transformaciones aplicadas a los datos
        pickle.dump(estimator, file)
    
    # Al salir del bloque 'with', el archivo se cierra automáticamente
    # El modelo queda guardado y puede ser cargado posteriormente

In [7]:
def load_estimator():
    """
    Esta función carga un modelo previamente guardado desde el archivo.
    Si no existe un modelo guardado, retorna None.
    
    Returns:
        object or None: El modelo cargado o None si no existe archivo
    """
    
    # Importamos las librerías necesarias
    import os      # Para verificar si existe el archivo
    import pickle  # Para deserializar el objeto guardado

    # VERIFICACIÓN DE EXISTENCIA DEL ARCHIVO
    # Verificamos si existe el archivo "estimator.pickle"
    # Si no existe, significa que no hay un modelo previamente guardado
    if not os.path.exists("estimator.pickle"):
        return None  # Retornamos None si no hay modelo guardado
    
    # PROCESO DE CARGA:
    # Si el archivo existe, lo abrimos en modo binario de lectura ("rb")
    with open("estimator.pickle", "rb") as file:
        # pickle.load() deserializa el objeto desde el archivo
        # Esto restaura completamente el modelo con:
        # - Todos los pesos y parámetros entrenados
        # - La configuración de hiperparámetros
        # - Las transformaciones de datos configuradas
        estimator = pickle.load(file)

    # Retornamos el modelo cargado, listo para hacer predicciones
    return estimator

In [8]:
# CELDA 7: FUNCIÓN PRINCIPAL DE ENTRENAMIENTO CON COMPARACIÓN DE MODELOS
def train_estimator(estimator):
    """
    Esta función entrena un modelo y lo compara con un modelo previamente guardado.
    Solo guarda el nuevo modelo si es mejor que el anterior.
    
    Args:
        estimator: El modelo a entrenar y evaluar
    """
    
    # Importamos las librerías necesarias
    from sklearn.linear_model import LinearRegression  # (No se usa en este contexto)
    from sklearn.metrics import mean_absolute_error    # Para calcular error absoluto medio

    # PASO 1: PREPARACIÓN DE DATOS
    # Cargamos los datos usando la función definida anteriormente
    data, target = load_data()

    # Dividimos los datos en conjuntos de entrenamiento y prueba
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,      # Características
        y=target,    # Variable objetivo
    )

    # PASO 2: ENTRENAMIENTO DEL NUEVO MODELO
    # Entrenamos el estimador con los datos de entrenamiento
    # fit() ajusta los parámetros del modelo a los datos
    estimator.fit(x_train, y_train)

    # PASO 3: COMPARACIÓN CON MODELO ANTERIOR (SI EXISTE)
    # Intentamos cargar un modelo previamente guardado
    best_estimator = load_estimator()

    # Si existe un modelo anterior, comparamos su rendimiento
    if best_estimator is not None:

        # Calculamos el error del modelo guardado anteriormente
        # mean_absolute_error mide la diferencia promedio entre predicciones y valores reales
        saved_mae = mean_absolute_error(
            y_true=y_test,                              # Valores reales de prueba
            y_pred=best_estimator.predict(x_test)       # Predicciones del modelo guardado
        )

        # Calculamos el error del modelo actual recién entrenado
        current_mae = mean_absolute_error(
            y_true=y_test,                       # Valores reales de prueba
            y_pred=estimator.predict(x_test)     # Predicciones del modelo actual
        )

        # DECISIÓN: ¿Cuál modelo es mejor?
        # Si el modelo guardado tiene menor error, lo mantenemos
        # Solo reemplazamos si el nuevo modelo es mejor
        if saved_mae < current_mae:
            estimator = best_estimator  # Mantenemos el modelo anterior (mejor)

    # PASO 4: GUARDAR EL MEJOR MODELO
    # Guardamos el modelo (ya sea el nuevo si es mejor, o el anterior si era mejor)
    save_estimator(estimator)

In [9]:
# CELDA 8: ENTRENAMIENTO DE REGRESIÓN LOGÍSTICA CON OPTIMIZACIÓN DE HIPERPARÁMETROS
def train_logistic_regression():
    """
    Esta función configura y entrena un modelo de Regresión Logística
    con búsqueda automática de los mejores hiperparámetros.
    """
    
    # Importamos el algoritmo de Regresión Logística
    from sklearn.linear_model import LogisticRegression

    # PASO 1: CREACIÓN DEL PIPELINE
    # Creamos un pipeline completo que incluye:
    # - Transformación de datos categóricos
    # - Selección de características
    # - Modelo de Regresión Logística
    pipeline = make_pipeline(
        estimator=LogisticRegression(
            max_iter=10000,    # Máximo 10,000 iteraciones para convergencia
            solver="saga"      # Algoritmo 'saga' que soporta regularización L1 y L2
        ),
    )

    # PASO 2: DEFINICIÓN DEL ESPACIO DE BÚSQUEDA DE HIPERPARÁMETROS
    # param_grid define todos los valores que queremos probar para cada parámetro
    param_grid = {
        # Número de características a seleccionar (de 1 a 10)
        "selectkbest__k": range(1, 11),
        
        # Tipo de regularización:
        # - "l1": Lasso (elimina características irrelevantes)
        # - "l2": Ridge (reduce magnitud de coeficientes)
        "estimator__penalty": ["l1", "l2"],
        
        # Fuerza de regularización (parámetro C):
        # - Valores pequeños (0.001): alta regularización
        # - Valores grandes (100): baja regularización
        "estimator__C": [0.001, 0.01, 0.1, 1, 10, 100],
    }
    
    # TOTAL DE COMBINACIONES A PROBAR:
    # 10 valores de k × 2 tipos de penalty × 6 valores de C = 120 combinaciones

    # PASO 3: CONFIGURACIÓN DE GRID SEARCH
    # Creamos el objeto que probará todas las combinaciones
    estimator = make_grid_search(
        estimator=pipeline,        # Pipeline a optimizar
        param_grid=param_grid,     # Parámetros a probar
        cv=5,                     # Validación cruzada de 5 folds
    )

    # PASO 4: ENTRENAMIENTO Y EVALUACIÓN
    # Entrenamos el modelo con búsqueda de hiperparámetros
    # Esta función también compara con modelos anteriores
    train_estimator(estimator)


# EJECUCIÓN: Entrenamos el modelo de Regresión Logística
train_logistic_regression()

In [10]:
def eval_metrics(
    y_train_true,    # Etiquetas reales del conjunto de entrenamiento
    y_test_true,     # Etiquetas reales del conjunto de prueba
    y_train_pred,    # Predicciones del modelo en entrenamiento
    y_test_pred,     # Predicciones del modelo en prueba
):
    """
    Esta función calcula métricas de rendimiento para evaluar la calidad
    del modelo tanto en entrenamiento como en prueba.
    
    Args:
        y_train_true: Etiquetas verdaderas de entrenamiento
        y_test_true: Etiquetas verdaderas de prueba  
        y_train_pred: Predicciones en entrenamiento
        y_test_pred: Predicciones en prueba
    
    Returns:
        tuple: (accuracy_train, accuracy_test, balanced_accuracy_train, balanced_accuracy_test)
    """
    
    # Importamos las métricas de evaluación de scikit-learn
    from sklearn.metrics import accuracy_score, balanced_accuracy_score

    # MÉTRICA 1: ACCURACY (EXACTITUD) 
    # Mide el porcentaje de predicciones correctas
    # Fórmula: (Predicciones correctas) / (Total de predicciones)
    
    # Accuracy en entrenamiento
    accuracy_train = round(accuracy_score(y_train_true, y_train_pred), 4)
    
    # Accuracy en prueba (el más importante para evaluar generalización)
    accuracy_test = round(accuracy_score(y_test_true, y_test_pred), 4)
    
    # MÉTRICA 2: BALANCED ACCURACY (EXACTITUD BALANCEADA)
    # Mejor métrica para datasets con clases desbalanceadas
    # Promedia la sensibilidad (recall) de cada clase
    # Es más robusta cuando hay diferentes cantidades de ejemplos por clase
    
    # Balanced accuracy en entrenamiento
    balanced_accuracy_train = round(
        balanced_accuracy_score(y_train_true, y_train_pred), 4
    )
    
    # Balanced accuracy en prueba
    balanced_accuracy_test = round(balanced_accuracy_score(y_test_true, y_test_pred), 4)

    # INTERPRETACIÓN DE RESULTADOS:
    # - Si accuracy_train >> accuracy_test: posible sobreajuste (overfitting)
    # - Si ambas son similares: buen equilibrio bias-varianza
    # - Balanced accuracy es más confiable en problemas de clasificación médica
    
    # Retornamos las cuatro métricas calculadas
    return (
        accuracy_train,
        accuracy_test,
        balanced_accuracy_train,
        balanced_accuracy_test,
    )

In [11]:
def report(
    estimator,                 # El modelo entrenado
    accuracy_train,           # Accuracy en entrenamiento
    accuracy_test,            # Accuracy en prueba
    balanced_accuracy_train,  # Balanced accuracy en entrenamiento
    balanced_accuracy_test,   # Balanced accuracy en prueba
):
    """
    Esta función genera un reporte visual del rendimiento del modelo
    mostrando las métricas de evaluación de forma organizada.
    
    Args:
        estimator: El modelo entrenado para mostrar su información
        accuracy_train: Exactitud en datos de entrenamiento
        accuracy_test: Exactitud en datos de prueba
        balanced_accuracy_train: Exactitud balanceada en entrenamiento
        balanced_accuracy_test: Exactitud balanceada en prueba
    """
    
    # FORMATO DEL REPORTE:
    
    # Línea 1: Muestra información del estimador (tipo de modelo y parámetros)
    print(estimator, ":", sep="")
    
    # Línea 2: Separador visual de 80 caracteres para mejorar legibilidad
    print("-" * 80)
    
    # Línea 3: Muestra Balanced Accuracy (métrica principal)
    # Formato: "Métrica: valor_prueba (valor_entrenamiento)"
    # El valor de prueba aparece primero porque es el más importante
    print(f"Balanced Accuracy: {balanced_accuracy_test} ({balanced_accuracy_train})")
    
    # Línea 4: Muestra Accuracy tradicional como métrica secundaria
    # Misma estructura: prueba primero, entrenamiento entre paréntesis
    print(f"         Accuracy: {accuracy_test} ({accuracy_train})")
    
    # INTERPRETACIÓN DEL FORMATO:
    # - El valor sin paréntesis es el rendimiento en datos NO VISTOS (prueba)
    # - El valor entre paréntesis es el rendimiento en datos de entrenamiento
    # - Si hay gran diferencia, puede indicar sobreajuste
    # - Balanced Accuracy se muestra primero por ser más robusta

In [12]:
def check_estimator():
    """
    Esta función carga el mejor modelo guardado y evalúa su rendimiento
    en los conjuntos de entrenamiento y prueba, mostrando un reporte completo.
    """
    
    # Importamos las librerías necesarias
    import pickle                                                    # Para cargar modelo
    import pandas as pd                                             # Para manipulación de datos
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # Métricas (no usadas aquí)

    # PASO 1: PREPARACIÓN DE DATOS
    # Cargamos el mismo dataset usado para entrenamiento
    data, target = load_data()

    # Creamos la misma división entrenamiento/prueba
    # IMPORTANTE: Usar la misma división para comparación justa
    x_train, x_test, y_train_true, y_test_true = make_train_test_split(
        x=data,      # Características
        y=target,    # Variable objetivo
    )

    # PASO 2: CARGA DEL MEJOR MODELO
    # Cargamos el modelo que fue guardado como el mejor
    # Este modelo ya tiene los hiperparámetros optimizados
    estimator = load_estimator()

    # PASO 3: GENERACIÓN DE PREDICCIONES
    # Hacemos predicciones en ambos conjuntos de datos
    
    # Predicciones en datos de entrenamiento
    # Útil para detectar sobreajuste si la diferencia con prueba es muy grande
    y_train_pred = estimator.predict(x_train)
    
    # Predicciones en datos de prueba
    # Esta es la métrica más importante: rendimiento en datos no vistos
    y_test_pred = estimator.predict(x_test)

    # PASO 4: CÁLCULO DE MÉTRICAS
    # Calculamos todas las métricas de evaluación
    (
        accuracy_train,           # Exactitud en entrenamiento
        accuracy_test,            # Exactitud en prueba
        balanced_accuracy_train,  # Exactitud balanceada en entrenamiento
        balanced_accuracy_test,   # Exactitud balanceada en prueba
    ) = eval_metrics(
        y_train_true,    # Etiquetas reales de entrenamiento
        y_test_true,     # Etiquetas reales de prueba
        y_train_pred,    # Predicciones en entrenamiento
        y_test_pred,     # Predicciones en prueba
    )

    # PASO 5: MOSTRAR REPORTE
    # Generamos un reporte visual con el rendimiento del modelo
    # estimator.best_estimator_ contiene el modelo con mejores hiperparámetros
    report(
        estimator.best_estimator_,    # Información del mejor modelo encontrado
        accuracy_train,               # Métricas de entrenamiento
        accuracy_test,                # Métricas de prueba
        balanced_accuracy_train,
        balanced_accuracy_test,
    )


# EJECUCIÓN: Evaluamos el modelo guardado
check_estimator()

Pipeline(steps=[('tranformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(dtype='int'),
                                                  ['thal'])])),
                ('selectkbest', SelectKBest(k=6)),
                ('estimator',
                 LogisticRegression(C=10, max_iter=10000, penalty='l1',
                                    solver='saga'))]):
--------------------------------------------------------------------------------
Balanced Accuracy: 0.6368 (0.8296)
         Accuracy: 0.6774 (0.8787)


In [13]:
# CELDA 12: ENTRENAMIENTO DE RED NEURONAL (MLP) CON OPTIMIZACIÓN DE HIPERPARÁMETROS
def train_mlp_classifier():
    """
    Esta función configura y entrena un Perceptrón Multicapa (MLP)
    con búsqueda automática de los mejores hiperparámetros.
    MLP es una red neuronal artificial para problemas de clasificación.
    """
    
    # Importamos el algoritmo de Red Neuronal (Multi-Layer Perceptron)
    from sklearn.neural_network import MLPClassifier

    # PASO 1: CREACIÓN DEL PIPELINE
    # Creamos un pipeline que incluye el preprocesamiento y la red neuronal
    pipeline = make_pipeline(
        estimator=MLPClassifier(
            max_iter=10000,    # Máximo 10,000 iteraciones para entrenamiento
                              # Las redes neuronales necesitan más iteraciones que otros algoritmos
        ),
    )

    # PASO 2: DEFINICIÓN DEL ESPACIO DE BÚSQUEDA DE HIPERPARÁMETROS
    # Para redes neuronales, optimizamos:
    param_grid = {
        # Número de características a seleccionar (igual que antes)
        "selectkbest__k": range(1, 11),
        
        # ARQUITECTURA DE LA RED NEURONAL:
        # hidden_layer_sizes define cuántas neuronas tiene la capa oculta
        # (h,) significa una sola capa oculta con h neuronas
        # Probamos desde 1 hasta 10 neuronas en la capa oculta
        "estimator__hidden_layer_sizes": [(h,) for h in range(1, 11)],
        
        # TASA DE APRENDIZAJE:
        # learning_rate_init controla qué tan rápido aprende la red
        # - Valores pequeños (0.0001): aprendizaje lento pero estable
        # - Valores grandes (1.0): aprendizaje rápido pero puede ser inestable
        "estimator__learning_rate_init": [0.0001, 0.001, 0.01, 0.1, 1.0],
    }
    
    # TOTAL DE COMBINACIONES A PROBAR:
    # 10 valores de k × 10 arquitecturas × 5 tasas de aprendizaje = 500 combinaciones
    # Esto es computacionalmente más costoso que la regresión logística

    # PASO 3: CONFIGURACIÓN DE GRID SEARCH
    # Creamos el objeto que probará todas las combinaciones
    estimator = make_grid_search(
        estimator=pipeline,        # Pipeline con red neuronal
        param_grid=param_grid,     # Parámetros específicos de MLP
        cv=5,                     # Validación cruzada de 5 folds
    )

    # PASO 4: ENTRENAMIENTO Y COMPARACIÓN
    # Entrenamos la red neuronal y la comparamos con modelos anteriores
    # Si la red neuronal es mejor, reemplazará al modelo guardado
    train_estimator(estimator)


# EJECUCIÓN DEL EXPERIMENTO COMPLETO:

# 1. Entrenamos la red neuronal con optimización de hiperparámetros
train_mlp_classifier()

# 2. Evaluamos inmediatamente el mejor modelo (puede ser MLP o el anterior)
# Esto nos permite comparar si la red neuronal mejoró el rendimiento
check_estimator()

c:\Users\Samuel Castaño\Desktop\Fundamentos Analtica 2025-1\Clases\Clase 10\PRE-23-selectkbest-para-clasificacion-SamuCasta\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:787: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


: 

: 